In [ ]:
import numpy as np
import pandas as pd
import csv

def get_cross_validation_file_indices(cross_validation_runs):
	init = [e for e in range(10)]
	training_eval = []
	for i in range(cross_validation_runs):
		init_temp = init[:]
		eval = init_temp[i]
		del init_temp[i]
		training = init_temp
		training_eval.append([training, eval])
	print("training_eval: ", training_eval)
	return training_eval
	
def get_training_eval_set(training_eval_index):
	"""
	Input: Number of files for training set
	Output: all training instances
	"""
	instances = [np.array([], dtype=np.float32), np.array([], dtype=np.float32)]
	for i in training_eval_index[0]:
		DATAPATH = "../sub_datasets/subset_"+str(i)+".csv"
		file_instances = get_instances_from_csv(DATAPATH, "all", False)	
		if instances[0].size == 0 and instances[1].size == 0:
			instances_train = file_instances
		else:
			instances_train = [np.vstack((instances[0], file_instances[0])), np.append(instances[1], file_instances[1])]
	
	DATAPATH = "../sub_datasets/subset_"+str(training_eval_index[1])+".csv"
	instances_eval = get_instances_from_csv(DATAPATH, "all", False)
	
	return [instances_train, instances_eval]

def get_instances_from_csv(datapath, train_or_eval, fixed):
	"""
	Input: datapath: datapath to be read from
	Output: list with first element np array with features, second element np array with respective class labels
	"""
	features = np.zeros(6)
	class_labels = np.zeros(1)
	
	df = pd.read_csv(datapath, sep=',',header=None)[1:]
	df = df.sample(frac=1).reset_index(drop=True)
	if train_or_eval == "train":
		features = df.values[1:500000, 0:6]
		class_labels = df.values[1:500000, 6]
	elif train_or_eval == "eval":
		features = df.values[500000:, 0:6]
		class_labels = df.values[500000:, 6]
	else:
		features = df.values[1:, 0:6]
		class_labels = df.values[1:, 6]
	
	print("features: ", features)
	print("class_labels: ", class_labels)
	
	return [features.astype(np.float), class_labels.astype(np.int)]

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from startEvaluation import *
from getData import *

cross_validation_runs = 10
xNN = 5

training_eval_index = get_cross_validation_file_indices(cross_validation_runs)

mean_accuracy = 0
confusion_matrix = 0
for i in range(cross_validation_runs):
	training_eval_instances = get_training_eval_set(training_eval_index[i])
	training_instances = training_eval_instances[0]
	x = training_instances[0]
	y = training_instances[1]
	
	model = KNeighborsClassifier(n_neighbors=xNN)
	model.fit(x, y)

	eval_instances = training_eval_instances[1]
	eval_x = eval_instances[0]
	eval_y = eval_instances[1]
	predicted = model.predict(eval_x)
	print("predicted_real: ", predicted[0:50])
	print("groundtruth: ", eval_y[0:50].ravel())

	test_eval = evaluation(eval_y.ravel(), predicted)
	test_eval.print_eval()
	mean_accuracy += test_eval.get_accuracy()
	confusion_matrix += test_eval.get_conf_matrix()
mean_accuracy /= cross_validation_runs
print("mean_accuracy: ", mean_accuracy)
print("confusion_matrix: ", confusion_matrix)

In [ ]:
from sklearn.naive_bayes import GaussianNB
from startEvaluation import *
from getData import *

def onlineNaiveBayes(batch_size):	
	j = 0
	while True:
		if j == 0:
			model.partial_fit(x[j:j+batch_size], y[j:j+batch_size], y)
		else:
			try:
				model.partial_fit(x[j:j+batch_size], y[j:j+batch_size])
			except:
				break
		j += batch_size

def offlineNaiveBayes():
	model.fit(x, y.ravel())

cross_validation_runs = 10
batch_size = 100

training_eval_index = get_cross_validation_file_indices(cross_validation_runs)

mean_accuracy = 0
confusion_matrix = 0
for i in range(cross_validation_runs):
	training_eval_instances = get_training_eval_set(training_eval_index[i])
	training_instances = training_eval_instances[0]
	x = training_instances[0]
	y = training_instances[1]

	model = GaussianNB()
	#onlineNaiveBayes(batch_size)
	offlineNaiveBayes()

	eval_instances = training_eval_instances[1]
	eval_x = eval_instances[0]
	eval_y = eval_instances[1]
	predicted = model.predict(eval_x)
	print("predicted_real: ", predicted[0:50])
	print("groundtruth: ", eval_y[0:50].ravel())
	
	test_eval = evaluation(eval_y.ravel(), predicted)
	test_eval.print_eval()
	mean_accuracy += test_eval.get_accuracy()
	confusion_matrix += test_eval.get_conf_matrix()
mean_accuracy /= cross_validation_runs
print("mean_accuracy: ", mean_accuracy)
print("confusion_matrix: ", confusion_matrix)